In [ ]:
import pandas as pd
import yfinance as yf
import time



url = "ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt"
df = pd.read_csv(url, sep="|")
# df = df.query("`Financial Status`=='N' and ETF=='N' and NextShares=='N' and `Test Issue`=='N'")
df = df[~df["Symbol"].str.contains('File Creation Time', case=False, na=False)]
df = df[df['ETF'] == 'Y']
nasdaq_tickers = df['Symbol'][~pd.isna(df['Symbol'])].tolist()

u = "https://www.nasdaqtrader.com/dynamic/symdir/otherlisted.txt"
df = pd.read_csv(u, sep="|")
df = df[df["ACT Symbol"]!="File Creation Time"]
# nyse = df.query("Exchange=='N' and ETF=='N' and `Test Issue`=='N'")
df = df.query("ETF=='Y'")
# pat = r"(preferred|depositary|warrant|right|unit|fund|trust|note|bond|etn|when issued|when distributed|spac)"; nyse_commonish = nyse[~nyse["Security Name"].str.contains(pat, case=False, na=False)]
# nyse_commonish.loc[:, "ACT Symbol"] = nyse_commonish["ACT Symbol"].str.replace(r'(\.[A-Za-z0-9]+|\$[A-Za-z0-9]+)$', '', regex=True)
nyse_tickers = sorted(df["ACT Symbol"].unique())



# temp = list(set([*nyse_tickers, *nasdaq_tickers]) - set(df_dividends['symbol']))
temp = [*nyse_tickers, *nasdaq_tickers]
ls_series = []

while temp:

    time.sleep(0.3)

    ticker = temp.pop()
    t = yf.Ticker(ticker)
    prev_close = t.fast_info["previous_close"]   # float

    # 배당/액션
    actions = t.actions

    if actions is None or actions.empty:
        continue

    try:
        longname = t.info['longName']

    except Exception as e:
        
        try:
            longname = t.info['shortName']
        
        except:
            continue

    actions = t.actions
    actions = actions.sort_index(ascending=False).reset_index()

    actions["symbol"] = ticker
    actions["longname"] = longname
    actions["close"] = prev_close
    actions["date_diff"] = ((actions["Date"] - actions["Date"].shift(-1)).astype(int)/86400000000000).astype(int)
    actions["rate"] = actions["Dividends"] / (actions["close"]) * 100


    ls_series.append(actions.iloc[0])

df_dividends = pd.DataFrame(ls_series)
df_dividends.to_json("_data/stocks/dividend_etf.json", orient="records", indent=2)
